## Quick File

In [1]:
import json
import seaborn as sns 
import matplotlib.pyplot as plt
import pandas as pd

mainfile = "../../result_2023_6_8.json"
maindata = []
with open(mainfile, 'r') as f:
  for entry in f:
      oneEntry = json.loads(entry)
      maindata.append(oneEntry)

In [114]:
def prettify_dataframe(df_orginal, setIndex = None):
    """
    Prettify specific columns of a DataFrame for readability.
    
    - 'Origin URLS': Lists are converted to comma-separated strings.
    - 'URL Leak Types': Tuples are converted to space-separated strings; multiple tuples to comma-separated strings.
    
    Parameters
    - df (panda df)

    Returns 
    - df: a pretty print read version of the df
    """
    df = df_orginal.copy()
    # Convert lists into comma-separated strings
    df['Origin URLS'] = df['Origin URLS'].apply(lambda x: ', '.join(map(str, x)))
    
    # Convert tuples into strings by joining their elements with spaces
    # df['URL Leak Types'] = df['URL Leak Types'].apply(lambda x: ', '.join([' '.join(map(str, t)) for t in x]))
    # df['URL Leak Types'] = df['URL Leak Types'].apply(lambda x: ', '.join([' '.join(t) for t in x]))

    if setIndex is not None:
        df = df.set_index(setIndex)
   
    return df

In [3]:
## Removing data entries where the json file has no report data saved:
# filtered_data = []
# for d in maindata: 
#   if d.get('report') != None: #not empty
#     if d.get('report').get('user_id') != None: 
#       filtered_data.append(d)

# filtered_data1_empty = []
# for d in maindata: 
#   if d.get('report') != None: #not empty
#     if d.get('report').get('initiator_domain') == None: 
#         filtered_data1.append(d)

filtered_data = []

for d in maindata:
    if d.get('report'):  # not empty
        if d.get('report').get('user_id') or d.get('report').get('initiator_domain'):
            filtered_data.append(d)


In [4]:
Leak_URLS = []
Origin_URLS = []
Leak_Types = []
for d in filtered_data: 
    rep = d['report']
    Origin_URLS.append(rep['initiator_domain'])
    if rep.get('url_leak_type') != None: 
        Leak_Types.append(tuple(rep['url_leak_type']))
    else: Leak_Types.append(("None"))
    if rep.get('tracker_info') != None: 
        if rep['tracker_info'].get('tracker').get('owner').get('url') != None: 
            #all reports with tracker info, have the following
            Leak_URLS.append(d['report']['tracker_info']['tracker']['owner']['url'])
        else: Leak_URLS.append("None")
    else: Leak_URLS.append("None")
        
df_urls = pd.DataFrame.from_dict({'Origin URLS': Origin_URLS,'Leak URLS': Leak_URLS, 'URL Leak Types': Leak_Types})

In [8]:
# Initializing the lists to store the extracted data
Leak_URLS = []
Origin_URLS = []
Leak_Types = []

# Loop through each item in the filtered_data list
for d in filtered_data:

    # Get the 'report' key from the current item. If it doesn't exist, default to an empty dictionary
    rep = d.get('report', {})

    # Append the value of 'initiator_domain' from the report to Origin_URLS.
    # If 'initiator_domain' doesn't exist, append None.
    Origin_URLS.append(rep.get('initiator_domain', None))
    
    # Append the 'url_leak_type' from the report as a tuple to Leak_Types.
    # If 'url_leak_type' doesn't exist, append a tuple with "None".
    url_leak_type = rep.get('url_leak_type')
    if url_leak_type is None:
        Leak_Types.append(("None",))
    else:
        Leak_Types.append(tuple(url_leak_type))


    # Traverse the nested dictionaries to get the desired URL.
    # If any of the keys don't exist, default to the next step or to "None" for the final value.
    # Starting with the initial dictionary
    tracker_info = rep.get('tracker_info', {})

    # Getting the nested dictionaries step by step, with safety checks
    tracker = tracker_info.get('tracker', {}) if tracker_info else {}
    owner = tracker.get('owner', {}) if tracker else {}
    tracker_url = owner.get('url', "None") if owner else "None"


    # Append the extracted tracker_url (or "None" if not found) to Leak_URLS
    Leak_URLS.append(tracker_url)

df_urls2 = pd.DataFrame.from_dict({'Origin URLS': Origin_URLS,'Leak URLS': Leak_URLS, 'URL Leak Types': Leak_Types})

In [9]:
# Check if the contents of the two DataFrames are the same
are_dataframes_equal = df_urls2.equals(df_urls)

# If they aren't the same, use the compare() method to find differences
if not are_dataframes_equal:
    differences = df_urls2.compare(df_urls)
    print(differences)

# Check if the data types of columns in the two DataFrames are the same
are_dtypes_equal = all(df_urls2.dtypes == df_urls.dtypes)
if not are_dtypes_equal:
    print("Data types differ between DataFrames")

# Combine both checks for a complete comparison
are_completely_equal = are_dataframes_equal and are_dtypes_equal

print(f"Are the DataFrames completely equal? {are_completely_equal}")


       URL Leak Types      
                 self other
222           (None,)  None
223           (None,)  None
232           (None,)  None
246           (None,)  None
247           (None,)  None
...               ...   ...
631239        (None,)  None
631268        (None,)  None
631269        (None,)  None
631270        (None,)  None
631274        (None,)  None

[286263 rows x 2 columns]
Are the DataFrames completely equal? False


In [117]:
# Group the df_urls2 dataframe by 'Origin URLS' and 'URL Leak Types' columns.
# For each group, the 'Leak URLS' are combined into a list.
grouped_df = df_urls2.groupby(['Origin URLS', 'URL Leak Types'])['Leak URLS'].apply(list).reset_index(name = "Leak URLS")

# Create a new column 'Appearances' which counts the number of times a leak URL appears (i.e., the length of 'Leak URLS' list).
grouped_df['Appearances'] = grouped_df['Leak URLS'].str.len()

# Modify the 'Leak URLS' column to contain unique values by converting the list into a set.
grouped_df['Leak URLS'] = grouped_df.apply(lambda row: set(row['Leak URLS']), axis = 1)

# Create a new column 'Count of Unique Leak URLS' which counts the number of unique leak URLs (i.e., the length of the set).
grouped_df['Count of Unique Leak URLS'] = grouped_df['Leak URLS'].str.len()

# Create a new column 'Count of Leak Types' which counts the number of leak types (based on the length of 'URL Leak Types' string).
grouped_df['Count of Leak Types'] = grouped_df['URL Leak Types'].str.len()

# Sort the grouped_df by 'Count of Unique Leak URLS' and 'Appearances' in descending order.
grouped_df.sort_values(by = ['Count of Unique Leak URLS', 'Appearances'], ascending = [False, False], inplace=True)

# Convert the 'Leak URLS' column to a string, remove 'None' values, and strip curly braces from the strings.
grouped_df['Leak URLS'] = grouped_df['Leak URLS'].astype(str).str.replace('None', '').str.strip('{}')

# Convert the 'URL Leak Types' column to a string, remove 'None' values,
grouped_df['URL Leak Types'] = grouped_df['URL Leak Types'].astype(str).str.replace('None', '')

# Create a copy of the df for further analysis
grouped_df_copy = grouped_df.copy()

# Final touches that are annoying to undo for the copy df...
# Set 'Origin URLS' as the index for the grouped_df.
# grouped_df = grouped_df.set_index('Origin URLS')

# Rearrange the order of columns in grouped_df for clarity.
# grouped_df = grouped_df[['URL Leak Types', 'Count of Leak Types', 'Appearances', 'Leak URLS', 'Count of Unique Leak URLS']]

# Create a copy for the next df
grouped_df.head(5)

,Origin URLS,URL Leak Types,Leak URLS,Appearances,Count of Unique Leak URLS,Count of Leak Types
99,https://quizlet.com/,"('lastname',)","'http://rhythmone.com', 'http://outbrain.com',...",5931,55,1
117,https://uchicagossecretplaces.wordpress.com/,"('lastname',)","'http://id5.io', 'http://magnite.com', 'http:/...",187,21,1
162,https://www.linkedin.com/,"('lastname',)","'http://freewheel.tv', 'http://id5.io', 'http:...",7991,9,1
133,https://www.citationmachine.net/,"('lastname',)","'http://newrelic.com', 'http://adobe.com', 'ht...",646,8,1
168,https://www.nytimes.com/,"('lastname',)","'http://chartbeat.com', 'http://iteratehq.com'...",197,7,1


## Create a dataframe where the Leak URLs are unique keys and the origin URLS are aggergated

In [105]:
# Convert the string representation of sets to actual sets
grouped_df_copy['Leak URLS'] = grouped_df_copy['Leak URLS'].apply(lambda x: eval(x))

# Explode the 'Leak URLS' column
df_exploded = grouped_df_copy.explode('Leak URLS').reset_index(drop=True)

# Group by 'URL Leak Types' and aggregate 'Origin URLs' into a list
grouped = df_exploded.groupby('Leak URLS').agg({
    'Origin URLS': lambda x: list(x.unique()),
    'URL Leak Types': lambda x: list(x.unique()),
    'Appearances': 'first',
    'Count of Unique Leak URLS': 'first',
    'Count of Leak Types': 'first'
}).reset_index()

# Count the number of empty entries in "Leak URLS"
empty_count = (grouped['Leak URLS'] == '').sum()
print(f"Number of empty entries in 'leak URLS': {empty_count}\n Entry removed.")

# Remove the rows with empty entries in "leak URLS"
# grouped = grouped[grouped['Leak URLS'] != '']

Number of empty entries in 'leak URLS': 1
 Entry removed.


In [116]:
prettify_dataframe(grouped, "Leak URLS").head(30)

,Origin URLS,URL Leak Types,Appearances,Count of Unique Leak URLS,Count of Leak Types
Leak URLS,,,,,
,"https://quizlet.com/, https://uchicagossecretp...","[('lastname',), ('preferredname',), ('lastname...",5931,55,1
http://33across.com,"https://quizlet.com/, https://uchicagossecretp...","[('lastname',)]",5931,55,1
http://adform.com,https://quizlet.com/,"[('lastname',)]",5931,55,1
http://adkernel.com,https://quizlet.com/,"[('lastname',)]",5931,55,1
http://adlightning.com,https://quizlet.com/,"[('lastname',), ('preferredname',)]",5931,55,1
http://adobe.com,"https://quizlet.com/, https://www.linkedin.com...","[('lastname',)]",5931,55,1
http://adotmob.com,https://quizlet.com/,"[('lastname',)]",5931,55,1
http://adtheorent.com,https://quizlet.com/,"[('lastname',)]",5931,55,1
http://adyoulike.com,"https://quizlet.com/, https://uchicagossecretp...","[('lastname',)]",5931,55,1


In [120]:
grouped["URL Leak Types"].to_csv("test.csv", encoding = "utf-8")

# TODO
Normalize counts for appearnces 
look into why just last name and where did that match
On the back end flag users for overly high lastname match and filter them
look at GET request for first name 